In [ ]:
import ee
import geemap
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


ee.Authenticate()
ee.Initialize(project="gsapp-map")

In [ ]:
dataset = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")

petermann_center = [-60.5, 80.5]
eqip_center = [-50.16800907246, 69.764798753]

# Petermann Glacier
# point = ee.Geometry.Point(-60.5, 80.5)

# Eqip Glacier - 69.76479875380193, -50.168009072460634
point = ee.Geometry.Point(-50.16800907246, 69.764798753)

region = point.buffer(100000)

image2017 = (
    dataset.filterDate("2017-01-01", "2018-01-01")
    .filterBounds(region)
    .mosaic()
    .reproject(crs="EPSG:3413", scale=100)
)
image2024 = (
    dataset.filterDate("2024-01-01", "2025-01-01")
    .filterBounds(region)
    .mosaic()
    .reproject(crs="EPSG:3413", scale=100)
)

# Visualize three axes of the embedding space as an RGB.
visParams = {min: -0.3, max: 0.3, "bands": ["A05", "A06", "A34"]}

Map = geemap.Map(center=[-51.0810317, 69.7276462], zoom=4)
Map.add_basemap("TopPlusOpen.Grey")

Map.addLayer(image2017, visParams, "2017 embeddings")
Map.addLayer(image2024, visParams, "2024 embeddings")

Map.centerObject(point, zoom=6)
Map.setOptions("SATELLITE")
Map

In [ ]:
# Clustering


def get_cmap_palette(n_clusters, cmap_name="viridis"):
    cmap = plt.get_cmap(cmap_name)
    colors = [mcolors.to_hex(cmap(i / (n_clusters - 1))) for i in range(n_clusters)]
    return colors


# Sample points for training
n_samples = 1000

training2017 = image2017.sample(
    region=region, scale=10, numPixels=n_samples, seed=100, geometries=True
)
training2024 = image2024.sample(
    region=region, scale=10, numPixels=n_samples, seed=100, geometries=True
)
combined_training = training2017.merge(training2024)
bands = list(
    set(image2017.bandNames().getInfo()) & set(image2024.bandNames().getInfo())
)

Map = geemap.Map(center=eqip_center, zoom=6)
Map.add_basemap("TopPlusOpen.Grey")

clusters = [6, 10, 12, 20]

for c in clusters:
    clusterer = ee.Clusterer.wekaKMeans(c).train(
        features=combined_training, inputProperties=bands
    )

    palette = get_cmap_palette(c, cmap_name="tab20")

    Map.addLayer(
        image2017.cluster(clusterer).visualize(min=0, max=c - 1, palette=palette),
        {},
        f"2017 : {c} Clusters",
    )
    Map.addLayer(
        image2024.cluster(clusterer).visualize(min=0, max=c - 1, palette=palette),
        {},
        f"2024 : {c} Clusters",
    )

Map.centerObject(point, zoom=9)
Map

In [ ]:
# Compute per-pixel Euclidean distance between embedding vectors
diff = image2024.select(bands).subtract(image2017.select(bands))
sq = diff.pow(2)
sum_sq = sq.reduce(ee.Reducer.sum())
euclidean_dist = sum_sq.sqrt().rename("euclidean_dist")

Map = geemap.Map(center=eqip_center, zoom=6)
Map.add_basemap("TopPlusOpen.Grey")

# Visualize the Euclidean distance (difference magnitude)
Map.addLayer(
    euclidean_dist,
    {"min": 0, "max": 0.5, "palette": ["black", "white"]},
    "Embedding Difference (Euclidean)",
)

Map.centerObject(point, zoom=9)
Map

In [ ]:
# Compute per-band absolute differences
band_diffs = []
mean_diffs = []

for band in bands:
    diff = image2024.select(band).subtract(image2017.select(band)).abs()
    band_diffs.append(diff.rename(band))

    # Reduce over region to get a mean distance
    mean_val = diff.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=region, scale=10, maxPixels=1e10
    ).get(band)
    mean_diffs.append((band, mean_val))

# Sort bands by mean difference and pick top 3
# Evaluate mean_diffs to get actual numbers
mean_diffs_eval = [(band, ee.Number(val).getInfo()) for band, val in mean_diffs]
mean_diffs_eval.sort(key=lambda x: x[1], reverse=True)
top3_bands = [band for band, val in mean_diffs_eval[:3]]

print("Top 3 bands with greatest difference:", top3_bands)
# Top 3 bands with greatest difference: ['A34', 'A05', 'A06']

In [ ]:
Map = geemap.Map(center=eqip_center, zoom=6)
Map.add_basemap("TopPlusOpen.Grey")

most_diff = ["A34", "A05", "A06", "A28"]

for band in most_diff:
    print("Visualizing band:", band)

    band_diff = (
        image2024.select(band).subtract(image2017.select(band)).rename(band + "_diff")
    )
    Map.addLayer(
        band_diff,
        {
            "min": 0,
            "max": 0.2,
            "palette": ["black", "white"],
        },
        f"{band} Difference (2024-2017)",
    )

Map.centerObject(point, zoom=9)

Map

In [ ]:
# Export data

# Select only the three bands with the most change for rgb
scale = 100  # meters per pixel
bands = ["A05", "A06", "A34"]
image2017_rgb = image2017.select(bands).reproject(crs="EPSG:3413", scale=scale)
image2024_rgb = image2024.select(bands).reproject(crs="EPSG:3413", scale=scale)

region_trimmed = point.buffer(60000).bounds()

geemap.ee_export_image(
    image2017_rgb,
    filename="data/embeddings_2017_rgb.tif",
    region=region_trimmed,
    scale=scale,
    crs="EPSG:3413",
)

geemap.ee_export_image(
    image2024_rgb,
    filename="data/embeddings_2024_rgb.tif",
    region=region_trimmed,
    scale=scale,
    crs="EPSG:3413",
)

geemap.ee_export_image(
    euclidean_dist,
    filename="data/euclidean_dist_2017_2024.tif",
    region=region_trimmed,
    scale=scale,
    crs="EPSG:3413",
)